In [15]:
from abc import ABC, abstractmethod
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [11]:
class TextFeatureExtractor(ABC):
    """Abstract class to describe a generic extraction of a feature starting from some text"""

    @abstractmethod
    def __init__(self, text, **kwargs):
        """
        Initialize the extractor with the raw text and other parameters which depend on the extractor
        Parameters:
        - text: raw input to extract feature from
        - **kwargs: dictionary containing specific extractor parameters
        """
        pass

    @abstractmethod
    def extract(self):
        """
        Extract the feature from the input text
        Return: an array containing the extracted features
        """
        pass

The purpose of the feature engineering is trying to add other features in addition to the word embeddings to provide more useful information for each word of the document to summarize

In [34]:
class TfIDfExtractor(TextFeatureExtractor):

    def __init__(self, text, **kwargs):
        """
        Parameters:
         text: raw document to extract td idf features
         **kwargs: function to use for tokenization
        """
        self.text = tokenize.sent_tokenize(text)
        self.vectorizer_tfidf = TfidfVectorizer()
        self.vectorizer_tf = TfidfVectorizer(use_idf=False)

    def extract(self):
        """TODO"""
        self.vectorizer_tfidf.fit_transform(self.text)
        x_tf = self.vectorizer_tf.fit_transform(self.text)
        # get word -> index vocabulary
        vocabulary = self.vectorizer_tf.vocabulary_

        x_idf = self.vectorizer_tfidf.idf_
        dict_idf = dict(zip(self.vectorizer_tfidf.get_feature_names(), x_idf))

        # conversion of continuous values of tf and idf into categorical ones using five categories
        x_tf_categorical = self.__convert_categorical(x_tf)

        # generation of pair of vectors containing tf idf category for each word in the document
        tf_words = []
        idf_words = []
        
        """
        for i,sentence in enumerate(self.text):
            for word in my_fun_tok(sentence):
                idf_words.append(dict_idf[word])
                tf_words.append(x_tf_categorical[i, vocabulary[word]])

        return tf_words, idf_words
        """
                
    def __convert_categorical(self, x):
        """
        Convert continuous tf and idf value into categorical one
        Parameters:
         x: matrix of continuous values to convert
        Return:
         converted categorical matrix
        """
        shape = x.shape
        x_categorical = np.empty(shape)

        # term frequency conversion
        for i in range(shape[0]):
            for j in range(shape[1]):
                if x[i,j] >= 0 and x[i,j] < 0.2:
                    x_categorical[i,j] = 0
                elif x[i,j] >= 0.2 and x[i,j] < 0.4:
                    x_categorical[i,j] = 1
                elif x[i,j] >= 0.4 and x[i,j] < 0.6:
                    x_categorical[i,j] = 2
                elif x[i,j] >= 0.6 and x[i,j] < 0.8:
                    x_categorical[i,j] = 3
                else:
                    x_categorical[i,j] = 4
        return x_categorical



document = """What's up fellas? So I got a patron supported that wanted me to talk about passive aggressive women like this is a category of women. All right, guys all women a passive-aggressive. All right guys, when a woman won't set a date with you or you know tell you will see or she cancels a date."""
extractor = TfIDfExtractor(document)
extractor.extract()

        

{'what': 30, 'up': 28, 'fellas': 6, 'so': 21, 'got': 7, 'patron': 16, 'supported': 22, 'that': 25, 'wanted': 29, 'me': 12, 'to': 27, 'talk': 23, 'about': 0, 'passive': 15, 'aggressive': 1, 'women': 35, 'like': 11, 'this': 26, 'is': 9, 'category': 4, 'of': 13, 'all': 2, 'right': 17, 'guys': 8, 'when': 31, 'woman': 34, 'won': 36, 'set': 19, 'date': 5, 'with': 33, 'you': 37, 'or': 14, 'know': 10, 'tell': 24, 'will': 32, 'see': 18, 'she': 20, 'cancels': 3}
